# Chapter 9
## Up and Running with Tensorflow
Refer to the book for other libraries and why tensorflow is sooo cooool! Anywho, hello tensorflow!

In [1]:
import tensorflow as tf

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

And thats it! Do note, **this does not perform any computation, even though it looks like it does**, it just creates a computation graph. Even the variables are not initialized yet! To evaluate this, you need to start a tf session and use it to initialize the varables and evaulate f.

In [2]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval

While this works, it's really tedious... Here's a simpler version of this!!!

In [3]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()

Note that a tensorflow program is split into two parts: the first part builds the computation graph (called the _construction phase_), and the second part runs it (called the _execution phase_).

## Managing Graphs
